In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import HRPOpt

tickers = ['MSFT', 'GOOG', 'JPM', 'XLE', 'GLD', 'SPY'] 

print("Downloading data...")
raw_data = yf.download(tickers, start='2020-01-01', end='2025-01-01')

if 'Adj Close' in raw_data.columns.levels[0]:
    data = raw_data['Adj Close']
else:
    data = raw_data['Close']

if 'SPY' not in data.columns:
    print("Error: SPY not downloaded.")
else:
    data = data.dropna()
    returns = data.pct_change().dropna()

    rebalance_period = 21 
    window_size = 252 

    hrp_portfolio_returns = []
    spy_benchmark_returns = []
    dates = []

    print("Running Walk-Forward Backtest with Energy Hedge...")

    for i in range(window_size, len(returns), rebalance_period):
        train_data = returns.iloc[i-window_size:i]
        test_data = returns.iloc[i:i+rebalance_period]
        
        train_assets = train_data.drop(columns=['SPY'])
        test_assets = test_data.drop(columns=['SPY'])
        
        hrp = HRPOpt(train_assets)
        hrp_weights = hrp.optimize()
        
        w_list = np.array([hrp_weights[t] for t in test_assets.columns])
        
        monthly_ret = np.dot(test_assets, w_list)
        hrp_portfolio_returns.extend(monthly_ret)
        
        spy_benchmark_returns.extend(test_data['SPY'])
        dates.extend(test_data.index)

    results_df = pd.DataFrame({
        'HRP Strategy': hrp_portfolio_returns,
        'S&P 500': spy_benchmark_returns
    }, index=dates)

    cumulative_returns = (1 + results_df).cumprod()

    strategy_ret = results_df['HRP Strategy'].mean() * 252
    strategy_vol = results_df['HRP Strategy'].std() * np.sqrt(252)
    strategy_sharpe = (strategy_ret - 0.04) / strategy_vol

    print(f"New HRP Sharpe Ratio: {strategy_sharpe:.2f}")

    plt.figure(figsize=(10, 5))
    plt.plot(cumulative_returns['HRP Strategy'], label=f'HRP (Diversified)', linewidth=2, color='#005EB8')
    plt.plot(cumulative_returns['S&P 500'], label='S&P 500 Benchmark', linestyle='--', color='gray', alpha=0.7)
    plt.title('Walk-Forward Analysis: HRP vs Benchmark')
    plt.ylabel('Growth of $1')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.savefig('Walk_Forward_Backtest.png', dpi=300)
    plt.show()